## Introduction

Why repeat analysis? When new members join a team or when members depart organizations, valuable information also leaves the organization due to jupyter Notebooks, word documents, or PDFs containing critical analysis existing on the departing members’ desktop. This issue of knowledge management causes many person-hours to be wasted. However, modern solutions allow analysis to persist beyond members' time at an organization. This project places this issue in the context of physics-aware machine learning of highly energetic material. It explicitly addresses data storage related to direct numerical simulations of a shock traveling through porous explosive material with a single void in its construction. When the shock occurs, chemical reactions cause the void to collapse and a denotation to occur, modeled using neural networks instead of direct numerical simulations. 

The specific goal of this project is to ingest the simulation dataset to present an analysis of how the state of the system changes over time before any machine learning. After reading the dataset, a dashboard will display the distribution of the fields of interest, elementary statistics (mean, median standard deviation) of the variables of interest, and more advanced analysis like a spatial gradient and an animation of the simulation. The current analysis of this dataset resides in a jupyter notebook with handpicked visualizations of a subset of the simulation data. The main contribution of this project is creating the free and open source implementation of an influx database for scientific machine learning concerning physics-informed machine learning.

## Background

### Physics Aware Recurrent Convolutional Neural Networks version 2
This research uses Physics Aware Recurrent Convolutional Neural Networks version 2 (PARCv2) [1] to predict how the energetic material changes over time once a sock is applied to the system. This process can be described formally as an advection-diffusion-reaction system and is modeled using a partial differential equation (PDE) given below. 
$$ \frac{\partial \textbf{x}}{\partial t} = k  \nabla \textbf{x} - \textbf{u} \cdot \Delta + \textbf{R}_{\textbf{x}}(\textbf{x},\textbf{u},\textbf{c})$$


With initial conditions,

$$ \textbf{u}(t=0) = \textbf{u}_{0} $$
$$ \textbf{x}(t=0) = \textbf{x}_{0} $$


In the above PDE, $\textbf{x}$ is the variable of interest: temperature, pressure, or microstructure at a position and time $ t$. $ k $ is the diffusivity coefficient. $ \textbf{u} $ is the velocity field and is typically decomposed to $ \textbf{u}_x $ and $\textbf{u}_y$ to represent the velocity in the $x$ and $y$ direction, respectively.   If $ \textbf{R}_{\textbf{x}} $  is equal to zero, then it is known as Burgers’ equation. Essentially, PARCv2 is trying to learn the next state of the system, which can be solved numerically.[1]

### Influx
InfluxData Inc created InfluxDB is a NoSQL database focused on storing and visualizing time series data. It stores time series data in a parquet format and has an official docker image at https://hub.docker.com/_/influxdb. The utility it provides is the ability to query time series data quickly. [2]


## Data

The primary data source is 200 direct numerical simulations (DNS) of highly energic material receiving a shock over time. Each file contains velocity, temperature, and pressure fields at a $(x,y)$ position in space over time. Each simulation is a npy file representing a 128 by 256-pixel image with each $(x,y)$ pixel representing the location and containing temperature, pressure, and microstructure values at time $t$ of energetic material.

Each simulation lasts for 20 to 40 nanoseconds. The temperature is measured in degrees kelvin and is valued in the range $[300, 5000]$. Velocity is measured in micrometers per nanosecond. Pressure is measured on gigapascals, and microstructure is expressed as in the range $[0,1]$. Each file is hosted in the Visual Intelligence Laboratory project folder on the University of Virginia High-Performance Computing HPC System, and in total, there are 30 GB of files.

 A fallback is using data at https://github.com/pdebench/PDEBench, an open-source repository of scientific machine-learning datasets. There is data for Burger's equation mentioned in the background, but it is a 93 GB file.[3]

## Potential Analysis

In data science, one should understand the problem before making an inference. Therefore, the project aims to produce a dashboard to present exploratory analysis of the simulation data before feeding the data to the neural network within PARCv2. The project will focus on ingesting the npy files post-initial processing into a time series database and automatically updating a dashboard with key visualizations to enable the exploratory data analysis of the simulation data. The dashboard will have a dropdown list to select a particular simulation and then show the elementary statistics and the distribution of the variables of interest. Additionally, the dashboard will allow the user to see the simulation play over time. This is shown in the GIF below. The images shown is the temperature, pressure, microstructure, velocity in the x and velocity and the y direction changing over time once a shock is applied to the energiectic material.  

![title](images/00_002.gif)

The figure below shows an example histogram of the pressure data for every value in the pressure field of a 64 x 128 image over 25 timesteps from a simulation. For pressure alone, there are 204,800 values in just one simulation.


![title](images/pressure_histogram.png)

## Challenges

The main challenge of this project will be scope. Many of the example visualizations rely on the data having the shape of a tensor. Thus, adapting existing code to a new structure or designing the database queries to get the data in the correct format will be a challenge. Fortunately, there is utility in producing easily repeatable and transparent exploratory data analysis beyond this class project. Therefore, I plan to continue this work well beyond the fall semester. Handling the multi-dimensional arrays stored in the npy files is another challenge. The files must be unpacked and read into the influx db one channel at a time. Another issue is that Influx is a NoSQL database, and I have no experience with this type of backend. The syntax is written in Flux, Influx DB’s scripting and query. The documentation at https://docs.influxdata.com/influxdb/v2/ seems to be robust, but I will judge its quality once I dive in. Finally, the true challenge is reading from the data source outside the HPC environment. There are over 30 GB of files. However, I will only use a few files to develop locally. At a high level, the steps to success for this project are defining a database schema for the influx tables, writing the npy file to the influx database, querying the influx database to populate visualizations, and adapting existing visualizations to this new structure. Reading some of the influx documentation reveals that the npy files will potentially have to be converted to CSV’s to facilitate populating the database, which would also be challenging.


# Project Check In 2

## Progress report 

summarizes the project and all of the progress made since the beginning of the semester. <br><br>
I have confirmed that the data is available under MIT License [4]. They are stored as npy files. Each file has the shape (timesteps, channels, height, width), where the number of channels is five, and the height and weight of the image is 128 x 256 pixels. The figure Interactive Visualization Mock-up is an example of the interactive plot that will be placed on the dashboard. 

### Interactive Visualization Mock up

![title](images/pressure_demo.png)

Data wrangling involved looping through each array channel and storing the timestep, height, and width in a data frame. The data frame is then converted to JSON for reading into the database—an example of the dataframe after data wrangling is given below.

Each dataframe will have on the order of five hundred thousand records. This is value was obtained using 15 as an example number of timestep and multiplying timesteps, height and width.

## Data summary

In plain language, describes your choices with regard to selecting features to keep from the raw data, the wrangling needed, and the number and types of databases chosen to store the data. <br><br>


Initially, influxDB was chosen as the backend database. The parameters necessary to make a connection to the database are as follows:

influx_password = os.getenv('DOCKER_INFLUXDB_INIT_PASSWORD') <br>
influx_user= os.getenv('DOCKER_INFLUXDB_INIT_USERNAME') <br>
influx_token= os.getenv('DOCKER_INFLUXDB_INIT_TOKEN') <br>
influx_org= os.getenv('DOCKER_INFLUXDB_INIT_ORG') <br>
influx_bucket= os.getenv('DOCKER_INFLUXDB_INIT_BUCKET') <br>

client = influxdb_client.InfluxDBClient(url="http://localhost:8086", username=influx_user, password=influx_password, token=influx_token, org=influx_org) <br><br>
I am currenttly having an issue with writing my data to the influxdb. The write example  below from the influx documentation works but when I adapt it to my data, nothing gets writen in the measurement bucket.

write_api = client.write_api(write_options=SYNCHRONOUS) <br>

p = influxdb_client.Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)<br>
write_api.write(bucket=influx_bucket, org=influx_org, record=p)<br>

Due to this issue, MongoDB will store the data. As stated in the previous section, each npy file has been converted to a dataframe to enable the creation of the dashboard's data visualizations. This data frame is then converted to JSON and subsequently read into the MongoDB. Five documents, one for each channel, are created for one simulation file. The structure of the json is as follows:

{ <br>
  &nbsp;&nbsp;&nbsp;&nbsp;"variable": "pressure",  <br>
  &nbsp;&nbsp;&nbsp;&nbsp;"file": "void_100",  <br>
  &nbsp;&nbsp;&nbsp;&nbsp;"Data": [  <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"timestep": 0, "x": 0, "y": 0, "value": 100.0},  <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"timestep": 0, "x": 0, "y": 1, "value": 200.0},  <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{"timestep": 0, "x": 0, "y": 2, "value": 300.0}  <br>
  ]} <br>

In the example json, variables can be temperature, pressure, microstructure, velocity (x) or velocity (y). The file key is the name of the npy file. The data key contains the information within the npy file. <br><br>
After successfuly creating the dashboard using the mongo database, experimentation with the influx database will continue as time allows.

Since this is a NoSql database the are no any relationships to document in an ERD. However, an example diagram of the mongoDB structure of each file is available at https://dbdocs.io/clarencew0083/parc_dash


As stated in the data section, each simulation lasts for 20 to 40 nanoseconds. The temperature is measured in degrees kelvin and is valued in the range $[300, 5000]$. Velocity is measured in micrometers per nanosecond. Pressure is measured on gigapascals, and microstructure is expressed as in the range $[0,1]$.

# Database documentation and visualizations 

the first draft of complete database documentation for each database you build for this project. The documentation for one database should include tables that list the name of every feature in the database, its data type, whether the feature is a primary key or a foreign key that links to a primary key in another table, and short notes that describe what the feature means. The documentation should also include an ER diagram if thereare multiple tables included in the database. You may construct all of this by hand, or you can use an automated tool such as https://dbdocs.io.

## References


[1] Nguyen, et al., “PARCv2: Physics-aware Recurrent Convolutional Neural Networks for Spatiotemporal Dynamics Modeling,” arXiv (Cornell University), Feb. 2024, doi: https://doi.org/10.48550/arxiv.2402.12503.
‌

[2] “InfluxDB | Real-time insights at any scale | InfluxData,” InfluxData, Sep. 23, 2024. https://www.influxdata.com/homepage/ (accessed Oct. 08, 2024).

‌[3] Takamoto, Makoto; Praditia, Timothy; Leiteritz, Raphael; MacKinlay, Dan; Alesiani, Francesco; Pflüger, Dirk; Niepert, Mathias, 2022, "PDEBench Datasets", https://doi.org/10.18419/darus-2986, DaRUS, V8

[4] Visual Intelligence Laboratory. (2024). PARCv2: Physics-aware Recurrent Convolutional Neural Networks for Spatiotemporal Dynamics Modeling [Data set]. Zenodo.

## Gen AI Statement

I used generative AI to assist with data wrangling and converting existing visualizations to plotly. Additionally, I used the spell check and editor feature of Grammarly.